# **Analysis of Heart Disease Mortality Data**

### **Samarth Tuli, Ndenko Fontem, Yi Zhu, Kimia Samieinejad**    


**Importance of Heart Disease Mortality Research**    

**What is coronary heart disease?**

**Tutorial Purpose**

The objective of this tutorial is to evaluate many factors that may positively or negatively affect the heart disease mortality rates of populations across different countries so that we can get a better understanding of what factors should be focused upon most by the research community to reduce the overall risk of heart disease-related deaths globally. Data science is the right tool to achieve this because it will allow us to deconstruct complex heart disease mortality data into specific insights and recommendations that can be used by heart disease research leaders and policymakers to take immediate action through a 5-stage pipeline: data collection and processing, exploratory data analysis and visualizations, analysis/hypothesis testing/use of ML models, and insights & policy decisions.

In [5]:
print("Hello World. Testing notebook appears on GitHub page.")

Hello World. Testing notebook appears on GitHub page.


In [6]:
print("Yi Zhu testing")

Yi Zhu testing


In [ ]:
print("Ndenko testing")

In [12]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

suicide_rates_all_countries = pd.read_csv('./data/suicide_rates_all_countries.csv')
suicide_rates_all_countries .head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2017,...,NaN,NaN,0.0,NaN,0.0,0 [0 – 0],NaN,NaN,EN,2022-12-08T05:00:00.000Z
1,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2017,...,NaN,NaN,0.0,NaN,0.0,0 [0 – 0],NaN,NaN,EN,2022-12-08T05:00:00.000Z
2,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2017,...,NaN,NaN,0.0,NaN,0.0,0 [0 – 0],NaN,NaN,EN,2022-12-08T05:00:00.000Z
3,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,VCT,Saint Vincent and the Grenadines,Year,2017,...,NaN,NaN,0.0,NaN,0.0,0 [0 – 0],NaN,NaN,EN,2022-12-08T05:00:00.000Z
4,SDGSUICIDE,Crude suicide rates (per 100 000 population),text,AMR,Americas,Country,VCT,Saint Vincent and the Grenadines,Year,2017,...,NaN,NaN,0.0,NaN,0.0,0 [0 – 0],NaN,NaN,EN,2022-12-08T05:00:00.000Z
